In [49]:
import os
from os.path import join as oj
import sys, time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data_pecarn
import data_psrc
import data
import train
import matplotlib.gridspec as grd
import viz
import style

# sns.set(style="black")
# plt.style.use('dark_background')
outcome_def = 'iai_intervention' # output
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# combined

In [50]:
dummy = True
df_pecarn = data_pecarn.get_data(dummy=dummy)
feat_names, pecarn_feat_names = data.get_feat_names(df_pecarn)

df_psrc = data_psrc.get_data(dummy=dummy)
feat_names, psrc_feat_names = data.get_feat_names(df_psrc)

computing pecarn preprocessing...
computing psrc preprocessing...


In [51]:
feat_names = [k for k in psrc_feat_names if k in pecarn_feat_names]
meta = [outcome_def, 'cv_fold', 'dset']
common_feats = feat_names + meta
df1 = df_pecarn[common_feats]
df2 = df_psrc[common_feats]
df = df1.append(df2) # pd.concat((df1, df2), axis=0)


df = df[df.cv_fold.isin(data.pecarn_train_idxs)] # only use train idxs
# don't include doubles for binary features
feat_names = [f for f in feat_names if not f.endswith('_no')]
# print(len(feat_names), feat_names)

# don't include race or 
df['AbdTenderDegree_Mild'] = df['AbdTenderDegree_unknown'] | df['AbdTenderDegree_Mild']
feat_names = [f for f in feat_names if not 'Race' in f
              and '_or_' not in f
              and not f == 'AbdTenderDegree_unknown']
print(len(feat_names), feat_names)

27 ['VomitWretch_yes', 'AbdDistention_unknown', 'DecrBreathSound_yes', 'RtCostalTender', 'AbdomenPain_yes', 'Hypotension', 'MOI_Motor vehicle collision', 'AbdDistention_yes', 'ThoracicTrauma_yes', 'CostalTender', 'AbdTenderDegree_Moderate', 'MOI_unknown', 'MOI_Object struck abdomen', 'AbdTrauma_yes', 'Age', 'VomitWretch_unknown', 'MOI_Bike collision/fall', 'AbdTenderDegree_Severe', 'MOI_Motorcycle/ATV/Scooter collision', 'InitSysBPRange', 'InitHeartRate', 'MOI_Pedestrian/bicyclist struck by moving vehicle', 'LtCostalTender', 'GCSScore', 'SeatBeltSign_yes', 'MOI_Fall from an elevation', 'AbdTenderDegree_Mild']


In [ ]:
corrs = df[feat_names].corr()
sns.clustermap(corrs, cmap=style.cmap_div)
plt.savefig('results/corrplot.png', dpi=300, bbox_inches='tight')